In [1]:
import json
import glob

Start by importing the data and splitting it by lines

In [2]:
#filename = "tehstone_s3.json"
filename = "season3.json"
#filename = "blaseball-log-season3.json"
#filename = "combined-blaseball-log-1596691883485.json"

In [3]:
raw = []
with open(filename, 'r') as f:
    raw += f.read().split("\n")
if(raw[-1] == ""):
    raw = raw[:-1]

In [4]:
json_lines = []
for line in raw:
    json_lines.append(json.loads(line))

How many lines did we get?

In [5]:
print(len(json_lines))

51519


In [7]:
def legalStateChange(old_state, new_state):
    # Ignore bad starting game states
    if(old_state["lastUpdate"] == 'Game over.' and new_state["lastUpdate"] != 'Game over.'):
        return None
    if(old_state["lastUpdate"] == ""):
        return None
    
    # Check for skips in player roster
    if(new_state["awayTeamBatterCount"] - old_state["awayTeamBatterCount"] == -1):
        return (5, f"awayTeamBatterCount decremented. " +
                f"{old_state['awayTeamBatterCount']} -> {new_state['awayTeamBatterCount']}")
    if(new_state["homeTeamBatterCount"] - old_state["homeTeamBatterCount"] == -1):
        return (5, f"homeTeamBatterCount decremented. " +
                f"{old_state['homeTeamBatterCount']} -> {new_state['homeTeamBatterCount']}")
    return None

In [37]:
def updateUniqueID(update):
    return (update["_id"], update["inning"], update["topOfInning"], update["atBatBalls"], update["atBatStrikes"], update["lastUpdate"])

In [38]:
updates_by_game = {}
count = 0
errors = []
for line in json_lines:
    for update in line["schedule"]:
        if update["_id"] not in updates_by_game:
            updates_by_game[update["_id"]] = [update]
            continue;
        
        last_update = updates_by_game[update["_id"]][-1]
        if(last_update == update):
            continue
        
        change = legalStateChange(last_update, update)
        if(change):
            count += 1
            # print(update["_id"], ":", change)
            errors.append((update["_id"], update["inning"], change[0], change[1], updateUniqueID(update)))
        updates_by_game[update["_id"]].append(update)

In [39]:
len(updates_by_game)

991

In [40]:
count

549

In [43]:
def inspectGameError(error):
    updates = updates_by_game[error[0]]
#    print("ERROR WAS", error[3])
    for update in updates:
        if(updateUniqueID(update) == error[4]):
            print(updateUniqueID(update))
#         else:
#             print("     ", end='')
#         if(error[2] == 0):
#             print(f"Inning: {update['inning']}, {update['lastUpdate']}")
#         elif(abs(update["inning"] - error[1]) < 2):
#             if(error[2] == 5):
#                 print(f"(home#, away#): {(update['homeTeamBatterCount'], update['awayTeamBatterCount'])}, {update['lastUpdate']}")

In [44]:
inspectGameError(errors[1])

('bdf9d5fa-c16d-40a4-8880-2aef09bb7b76', 3, True, 0, 0, 'Joshua Butt gets caught stealing third base.')


In [45]:
for error in errors:
    inspectGameError(error)

('b3003ea4-fc12-4da0-bee4-f6fd8aaa92f5', 3, True, 0, 0, 'Reese Clark gets caught stealing third base.')
('bdf9d5fa-c16d-40a4-8880-2aef09bb7b76', 3, True, 0, 0, 'Joshua Butt gets caught stealing third base.')
('13ed042b-b972-4d4b-bf69-d27c251c5c75', 4, True, 0, 0, 'Elvis Figueroa gets caught stealing third base.')
('b8897596-7f3a-4704-acab-0a699349c356', 6, True, 0, 0, 'Sutton Picklestein gets caught stealing second base.')
('0394d25f-bccb-4b67-87f1-39085dec30f5', 8, True, 0, 0, 'Castillo Turner gets caught stealing third base.')
('0394d25f-bccb-4b67-87f1-39085dec30f5', 8, False, 0, 0, 'Taiga Quitter gets caught stealing third base.')
('bdf9d5fa-c16d-40a4-8880-2aef09bb7b76', 8, True, 0, 0, 'Dickerson Morse gets caught stealing third base.')
('d7370b77-5e41-44af-b8e6-e385e64521a0', 0, False, 0, 0, 'Hendricks Rangel gets caught stealing third base.')
('e2eef6ff-4f99-4916-ab49-34da6614bbaf', 7, True, 0, 0, 'Workman Gloom gets caught stealing third base.')
('3d3bba3d-ef75-463b-8764-4036d714

('8f8e729d-6f0b-47dc-9b00-ba7c73006a52', 6, False, 0, 0, 'York Silk gets caught stealing third base.')
('5cabbde2-fe76-40f3-baf1-2c65577996e3', 1, False, 0, 0, 'August Mina gets caught stealing third base.')
('e07d5f61-9740-438c-ba3a-2876fed70e44', 4, False, 0, 0, 'Stephens Lightner gets caught stealing third base.')
('23389b24-22f8-4213-ba74-7f164077b718', 5, True, 0, 0, 'Dominic Marijuana gets caught stealing third base.')
('73e901f7-55eb-45d4-9f9a-2bebf17ac2ba', 8, True, 0, 0, 'Kennedy Meh gets caught stealing third base.')
('53dfb746-c054-4978-bbc0-bcaa5d9fb197', 7, True, 0, 0, 'Castillo Turner gets caught stealing third base.')
('b79b30e9-2125-4a26-ade6-e08b32929bbf', 2, True, 0, 0, 'Rodriguez Internet gets caught stealing third base.')
('15b63abb-766b-4441-90ec-f4dc687a3384', 3, True, 0, 0, 'Axel Cardenas gets caught stealing third base.')
('3423fcc2-4e1c-4c4e-9b4a-9f2a68498dc6', 3, True, 0, 0, 'Marco Stink gets caught stealing third base.')
('d5faf464-61bf-41ea-b2c8-9db17fc61c98